In [10]:
import dash
import dash_mantine_components as dmc
from dash import html, dcc, Input, Output, State
import plotly.graph_objects as go
import plotly.express as px

import webbrowser
import pandas as pd

from collections import Counter

In [3]:
df = pd.read_csv("combined_processed.csv")

In [11]:
Counter(list(df["template_type"]))

Counter({'Mirror': 1381, 'XL+U': 1800, 'Other': 91, 'Jebus': 95, 'Duel': 11})

In [15]:
(1381+1800)/3378

0.9416814683244523

In [18]:
df

,player_one,score_player_one,player_two,score_player_two,winner,template,player_one_hero,player_two_hero,bidding_amount,ending_day,created_at,turns_played,player_one_town,player_one_colour,player_two_town,player_two_colour,template_type
0,HawkWRz,0.0,Vertr0X,1.0,Vertr0X,h3dm1/3,gundula,crag hack,900,124,19/05/2021 08:21,11.0,stronghold,red,stronghold,blue,Mirror
1,Bartt666,0.0,BOARtek,1.0,BOARtek,h3dm1/3,giselle,aeris,900,143,20/05/2021 14:49,24.0,rampart,red,rampart,blue,Mirror
2,BOARtek,0.0,GALAXER00,1.0,GALAXER00,mt_Firewalk,octavia,xyron,-5500,136,21/05/2021 14:44,20.0,inferno,red,inferno,blue,Mirror
3,Fafhrd,1.0,GALAXER00,0.0,Fafhrd,h3dm1/3,leena,jeremy,-900,122,21/05/2021 20:33,9.0,cove,red,cove,blue,Mirror
4,Ardian,1.0,Deextereek,0.0,Ardian,h3dm1/3,eovacius,anabel,700,127,21/05/2021 20:38,14.0,cove,red,cove,blue,Mirror
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3373,mirek_swirek,1.0,Shihtzu,0.0,mirek_swirek,6lm10a/tp,tyraxor,anabel,3600,123,26/02/2023 11:24,10.0,stronghold,red,cove,blue,XL+U
3374,Sc@neR,0.0,Jabolpunk,1.0,Jabolpunk,6lm10a/tp,tyraxor,thant,3000,134,26/02/2023 19:52,18.0,stronghold,red,necropolis,blue,XL+U
3375,Vens,1.0,GoldenCrow,0.0,Vens,6lm10a/tp,tyraxor,valeska,-5400,127,27/02/2023 19:34,14.0,stronghold,blue,castle,red,XL+U
3376,rybson1,1.0,BOARtek,0.0,rybson1,mt_Andromeda,pasis,gelare,800,131,27/02/2023 21:22,15.0,conflux,blue,conflux,red,Mirror


In [19]:
df.columns

Index(['player_one', 'score_player_one', 'player_two', 'score_player_two',
       'winner', 'template', 'player_one_hero', 'player_two_hero',
       'bidding_amount', 'ending_day', 'created_at', 'turns_played',
       'player_one_town', 'player_one_colour', 'player_two_town',
       'player_two_colour', 'template_type'],
      dtype='object')

In [5]:
# Having two graphs with a dropdown test

app = dash.Dash()   
df = pd.read_csv("combined_processed.csv")

df["absolute_bidding_amount"] = df["bidding_amount"].abs()

def create_graph():
    fig = go.Figure(px.histogram(df, x = "template"))
    fig.update_layout(title = 'Map usage', xaxis_title = 'Map', yaxis_title = 'Amount', #modebar_remove=['zoom', 'pan'],
                     xaxis={'fixedrange':True}, yaxis={'fixedrange':True})
    return fig  

def create_graph2():
    fig = go.Figure(px.histogram(df, x = "absolute_bidding_amount"))
    fig.update_layout(title = 'Bidding', xaxis_title = 'Bid', yaxis_title = 'Presence')
    return fig 


@app.callback(
    Output('dd-output-container', 'children'),
    Input('demo-dropdown', 'value')
)
def update_output(value):
    return f'You have selected {value}'
 
app.layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'Title', 
            style = {'textAlign':'center','marginTop':40,'marginBottom':40}),
    dcc.Graph(id = 'plot1', figure = create_graph()),
    dcc.Graph(id = 'plot1', figure = create_graph2()),
    dcc.Dropdown(['NYC', 'MTL', 'SF'], 'NYC', id='demo-dropdown'),
    html.Div(id='dd-output-container')
    ])

In [6]:
# Setup for a search function in the current graph
app = dash.Dash(__name__)   # Initialising dash app
df = pd.read_csv("combined_processed.csv")

app.layout = html.Div([dcc.Graph(id="interactive graph"),
                       dcc.Dropdown(list(df.columns), value = "player_one", id="interactive dropdown"),
                       dcc.Input(id="searcher input", value = "SDK_Ravciozo", type="text"),
                       html.Div(id="searcher output", children="text")])

@app.callback(
    Output("interactive graph", "figure"),
    Input("interactive dropdown", "value"))
def update_figure(column):
    print(column)
    fig = go.Figure(px.histogram(df, x = column))
    fig.update_layout(title = f'{column} Histogram', xaxis_title = column, yaxis_title = 'Amount')

    return fig

@app.callback(
    Output("searcher output", "children"),
    Input("searcher input", "value"),
    Input("interactive dropdown", "value"))
def update_figure(search, column):
    res = str(sum(df[column] == search))
    return f"The amount of {column} for {search} is {res}"

In [7]:
# Interactive Scatterplot

app = dash.Dash(__name__)   # Initialising dash app
df = pd.read_csv("combined_processed.csv")

df["absolute_bidding_amount"] = df["bidding_amount"].abs()

app.layout = html.Div([dcc.Graph(id="interactive graph"),
                       dcc.Dropdown(list(df.columns), value = "score_player_one", id="first dropdown"),
                       dcc.Dropdown(list(df.columns), value = "score_player_one", id="second dropdown")])

@app.callback(
    Output("interactive graph", "figure"),
    Input("first dropdown", "value"),
    Input("second dropdown", "value"))
def update_figure(column1, column2):
    fig = go.Figure(px.scatter(df, x = column1, y = column2, opacity=0.5))
    fig.update_layout(title = f'{column1} vs {column2}', xaxis_title = column1, yaxis_title = column2)

    return fig

In [8]:
# Interactive Heatmap

app = dash.Dash(__name__)   # Initialising dash app
df = pd.read_csv("combined_processed.csv")

df["absolute_bidding_amount"] = df["bidding_amount"].abs()

app.layout = html.Div([dcc.Graph(id="interactive graph"),
                       dcc.Dropdown(list(df.columns), value = "score_player_one", id="first dropdown"),
                       dcc.Dropdown(list(df.columns), value = "score_player_one", id="second dropdown")])

@app.callback(
    Output("interactive graph", "figure"),
    Input("first dropdown", "value"),
    Input("second dropdown", "value"))
def update_figure(column1, column2):
    fig = go.Figure(px.density_heatmap(df, x = column1, y = column2))
    fig.update_layout(title = f'{column1} vs {column2}', xaxis_title = column1, yaxis_title = column2)

    return fig

In [9]:
# Interactive Scatterplot ++ (with filtering)

app = dash.Dash(__name__)   # Initialising dash app
df = pd.read_csv("combined_processed.csv")

df["absolute_bidding_amount"] = df["bidding_amount"].abs()

app.layout = html.Div([dcc.Graph(id="interactive graph"),
                       dcc.Dropdown(list(df.columns), value = "score_player_one", id="first dropdown"),
                       dcc.Dropdown(list(df.columns), value = "score_player_one", id="second dropdown"),
                       dmc.Checkbox(label = "filter", checked = False, radius = "xl", id="filter checkbox"),
                       html.Div([dcc.Dropdown(list(df.columns), value = "score_player_one", id="filter dropdown"),
                                 dcc.Dropdown(sorted(list(set(df["score_player_one"]))), 
                                              value = sorted(list(set(df["score_player_one"])))[0], id="filter dropdown2")],
                                style = {"display": "block"})])

@app.callback(
    Output("interactive graph", "figure"),
    Input("first dropdown", "value"),
    Input("second dropdown", "value"),
    Input("filter checkbox", "checked"),
    Input("filter dropdown", "value"),
    Input("filter dropdown2", "value"))
def update_figure(column1, column2, to_filter, filter1, filter2):
    if to_filter: new_df = df[df[filter1] == filter2]
    else: new_df = df
    fig = go.Figure(px.scatter(new_df, x = column1, y = column2, opacity=0.5))
    fig.update_layout(title = f'{column1} vs {column2}', xaxis_title = column1, yaxis_title = column2)

    return fig

@app.callback(
    Output("filter dropdown", "style"),
    Output("filter dropdown2", "style"),
    Input("filter checkbox", "checked"))
def show_filters(checkbox):
    if checkbox: result = "block"
    else: result = "none"
    return {"display": result}, {"display": result}

@app.callback(
    Output("filter dropdown2", "options"),
    Output("filter dropdown2", "value"),
    Input("filter dropdown", "value"))
def update_second_filter(column):
    selection = sorted(list(set(df[column])))
    return selection, selection[0]

In [ ]:
webbrowser.open('http://127.0.0.1:8050/')
app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
